### Import the necessary libraries

In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from helpers.utils import load_data

### Load the datasets

In [3]:
traffic, stations, mappings = load_data()

### Data Cleaning

Let's check if there are any duplicate rows in traffic dataset

In [5]:
duplicates = traffic[traffic.duplicated()]
print(f"Traffic dataset contains {len(duplicates)} duplicate rows")
duplicates.head()

Traffic dataset contains 750678 duplicate rows


,date,day_of_data,day_of_week,direction_of_travel,fips_state_code,functional_classification,month_of_data,station_id,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,...,traffic_volume_counted_after_1400_to_1500,traffic_volume_counted_after_1500_to_1600,traffic_volume_counted_after_1600_to_1700,traffic_volume_counted_after_1700_to_1800,traffic_volume_counted_after_1800_to_1900,traffic_volume_counted_after_1900_to_2000,traffic_volume_counted_after_2000_to_2100,traffic_volume_counted_after_2100_to_2200,traffic_volume_counted_after_2200_to_2300,traffic_volume_counted_after_2300_to_2400
9440,2015-11-02,2,2,1,54,3R,11,000016,25,23,...,136,157,176,238,210,130,77,53,45,28
12434,2015-01-21,21,4,5,29,2U,1,004913,49,26,...,351,432,887,1407,1546,1015,377,243,193,115
14835,2015-08-09,9,1,7,54,4U,8,000045,23,20,...,260,300,200,187,219,159,136,108,45,34
14888,2015-04-25,25,7,1,13,3U,4,000222,27,20,...,252,258,265,258,213,172,122,93,75,50
16694,2015-04-06,6,2,3,39,1R,4,000155,184,121,...,893,923,932,898,731,573,476,373,323,230


Given that it is very unlikely for a data row to be exactly identical to another for all of its 32 columns (including the traffic volume for all the time intervals). It is highly likely that these data rows are genuine cases of duplicate entries, arising from errors made from the data collection/collation process. As such, let's drop these duplicate rows from our subsequent analysis.

In [6]:
traffic = traffic.drop_duplicates(ignore_index = True)
print(f"Number of rows left in unduplicated traffic dataset: {len(traffic)}")

Number of rows left in unduplicated traffic dataset: 6389713


Let's also check for invalid values in the traffic volume columns (i.e. negative values)

In [7]:
start_column = "traffic_volume_counted_after_0000_to_0100"
end_column = "traffic_volume_counted_after_2300_to_2400"

invalid = traffic[(traffic.loc[:, start_column: end_column] < 0).any(axis=1)]
print(f"There are {len(invalid)} rows containing negative values for traffic volume")
invalid.head()

There are 851 rows containing negative values for traffic volume


,date,day_of_data,day_of_week,direction_of_travel,fips_state_code,functional_classification,month_of_data,station_id,traffic_volume_counted_after_0000_to_0100,traffic_volume_counted_after_0100_to_0200,...,traffic_volume_counted_after_1400_to_1500,traffic_volume_counted_after_1500_to_1600,traffic_volume_counted_after_1600_to_1700,traffic_volume_counted_after_1700_to_1800,traffic_volume_counted_after_1800_to_1900,traffic_volume_counted_after_1900_to_2000,traffic_volume_counted_after_2000_to_2100,traffic_volume_counted_after_2100_to_2200,traffic_volume_counted_after_2200_to_2300,traffic_volume_counted_after_2300_to_2400
22298,2015-03-08,8,1,3,51,5R,3,130018,10,7,...,247,270,270,238,162,113,49,43,19,2
32468,2015-03-08,8,1,3,51,4R,3,150326,14,5,...,96,78,58,71,75,63,60,21,25,16
43542,2015-03-08,8,1,5,51,1R,3,781229,8,-1,...,40,50,44,36,22,26,28,18,16,3
45564,2015-03-08,8,1,3,51,1U,3,040766,667,-1,...,2733,2899,2886,2759,2709,2161,1698,1295,877,602
55491,2015-03-08,8,1,3,51,1U,3,090122,427,-1,...,689,682,704,678,586,530,451,294,210,135


Let's inspect what the negative values are and their respective counts

In [8]:
invalid_traffic_volume = invalid.loc[:, start_column: end_column].values

negative_values, counts = np.unique(
    invalid_traffic_volume[invalid_traffic_volume < 0],
    return_counts = True
)

print("Value: Count")
for value, count in zip(negative_values, counts):
    print(f"{str(value).rjust(5, ' ')}: {count}")

Value: Count
-3061: 1
   -1: 1448


In most cases (1448 out of 1449), -1 was the negative value. These negative values could be because the sensors were down for that particular time interval. Let's assume that the negative values were indeed due to the down sensors. In such a scenario, no values would be recorded for traffic volume (i.e. traffic volume = 0). As such, let's replace the negative values with 0.

In [9]:
replacements = {col: {value: 0 for value in negative_values} for col in traffic.columns[-24:]}
traffic = traffic.replace(replacements)

### Caveats

Let's check that in general, the traffic dataset does contain 365 days (a year) of data

In [10]:
print(f"Number of unique dates in traffic dataset: {len(traffic.date.unique())}")

Number of unique dates in traffic dataset: 365


For the purpose of subsequent analysis in caveats, let's define a unique station based on its fips_state_code, station_id, direction_of_travel and function_classification. What this means is that so long as two stations have different fips_state_code, station_id, direction_of_travel or function_classification, they are considered as two unique stations.

Let's check if evey unique station has 365 days (a year) worth of data

In [11]:
columns = ["date", "fips_state_code", "station_id", "functional_classification", "direction_of_travel"]
group_columns = ["fips_state_code", "station_id", "direction_of_travel", "functional_classification"]

num_unique_dates = traffic[columns] \
.groupby(group_columns) \
.agg("nunique") \
.reset_index(drop=False) \
.rename(columns = {"date": "num_unique_dates"})

num_unique_dates

,fips_state_code,station_id,direction_of_travel,functional_classification,num_unique_dates
0,01,000002,3,3R,325
1,01,000002,7,3R,324
2,01,000004,3,1R,330
3,01,000004,7,1R,330
4,01,000006,1,4R,287
...,...,...,...,...,...
12369,56,0169NE,5,3R,347
12370,56,0169NW,1,4R,347
12371,56,0169NW,5,4R,347
12372,56,RP0369,1,1U,359


From the table above, it seems that not all the unique stations have 365 days of recorded data. Let's take a look at the box plot for num_unique_dates.

In [12]:
num_unique_dates[["num_unique_dates"]].describe()

,num_unique_dates
count,12374.000000
mean,301.745919
std,87.625571
min,1.000000
25%,286.000000
50%,342.000000
75%,360.000000
max,365.000000


In [42]:
# It appears that not every station has all 365 days of its data recorded
# Let's take a look at the summary statistics for num_unique_dates
num_unique_dates[["num_unique_dates"]].describe()

,num_unique_dates
count,12374.000000
mean,301.745919
std,87.625571
min,1.000000
25%,286.000000
50%,342.000000
75%,360.000000
max,365.000000


In [43]:
# Let's sort the data by num_unique_dates
num_unique_dates.sort_values("num_unique_dates").head(100)

,fips_state_code,station_id,direction_of_travel,functional_classification,num_unique_dates
4766,22,000057,5,4R,1
4765,22,000057,1,4R,1
1347,06,043340,3,1U,1
11584,54,000090,3,1U,1
1511,06,075020,3,1U,1
...,...,...,...,...,...
839,04,101249,1,2U,24
838,04,101247,5,2U,24
130,01,000156,1,1U,25
9162,47,000235,3,1U,25


In [44]:
# Let's find the number of unique stations per state
# For our analysis, a unique station is defined based on its station_id, direction_of_travel and functional_classification
# i.e. two stations with the same station_id but different direction_of_travel is considered as 2 unique stations
num_stations = num_unique_dates.groupby(
    ["fips_state_code", "station_id", "direction_of_travel", "functional_classification"]) \
    .agg("count") \
    .reset_index() \
    .rename(columns = {"num_unique_dates": "num_unique_stations"})

num_unique_stations = num_stations[["fips_state_code", "num_unique_stations"]] \
                      .groupby("fips_state_code") \
                      .sum() \
                      .reset_index()
num_unique_stations

,fips_state_code,num_unique_stations
0,01,296
1,02,210
2,04,526
3,05,115
4,06,570
5,08,212
6,09,75
7,10,168
8,11,4
9,12,545


In [45]:
# There is a large spread in terms of the number of unique stations in each state as seen from below
num_unique_stations.describe()

,num_unique_stations
count,51.000000
mean,242.627451
std,166.828650
min,4.000000
25%,137.500000
50%,195.000000
75%,311.500000
max,961.000000


In [62]:
# Let's check if there is a correlation between the annual traffic volume and the number of unique stations in each state
# Intuitively, if a state has more unique stations, its annual traffic volume should also be higher, since it has more
# sensors that are tracking the traffic volume
# If our hypothesis is true, then annual traffic volume by state is not an accurate metric to measure how heavy the traffic is
# in a state, as the high traffic volume might just be because of the large number of sensors used for tracking
# In such a scenario, annual traffic volume normalized by the number of unique stations might be a more accurate metric to use.
start_column = "traffic_volume_counted_after_0000_to_0100"
end_column = "traffic_volume_counted_after_2300_to_2400"
traffic["total_daily_traffic_volume"] = traffic \
                                        .loc[:, start_column: end_column] \
                                        .sum(axis=1)

traffic_state = traffic[["fips_state_code", "total_daily_traffic_volume"]] \
                .groupby("fips_state_code") \
                .sum() \
                .reset_index() \
                .rename(columns = {"total_daily_traffic_volume": "annual_traffic_volume"})
traffic_state

,fips_state_code,annual_traffic_volume
0,01,1461509905
1,02,318138924
2,04,5600393206
3,05,278436058
4,06,6670057776
5,08,964145461
6,09,500556105
7,10,534104309
8,11,59205437
9,12,4412694931


In [71]:
# Merge num_unique_stations with traffic_state
merge_station_state = num_unique_stations.merge(traffic_state, on="fips_state_code")
fig = px.scatter(merge_station_state, x="num_unique_stations", y="annual_traffic_volume",
           color="fips_state_code", trendline="ols", trendline_scope="overall",
           hover_data=["fips_state_code"], trendline_color_override="black",
           labels={"num_unique_stations": "No of Unique Stations",
                   "annual_traffic_volume": "Annual Traffic Volume"})
fig

In [81]:
# Based on the graph above and the R^2 value = 0.769 (can be found by hovering over the trendline in the graph), we can  conclude
# that there is indeed a strong positive correlation between num_unique_stations and the annual traffic volume
# As such, it might be prudent to measure how heavy traffic is in a state using the metric: annual traffic volume by state
# normalized by the number of unique stations  

In [89]:
# Let's visualize the annual traffic volume by state
fips_state_abb = mappings.get("fips_state_abb")
fips_state_full = mappings.get("fips_state_full")

merge_station_state["fips_state_abb"] = merge_station_state.fips_state_code.apply(lambda x: fips_state_abb[x])
merge_station_state["fips_state_full"] = merge_station_state.fips_state_code.apply(lambda x: fips_state_full[x])

merge_station_state["norm_annual_traffic_volume"] = merge_station_state.annual_traffic_volume / merge_station_state.num_unique_stations

merge_station_state

,fips_state_code,num_unique_stations,annual_traffic_volume,fips_state_abb,fips_state_full,norm_annual_traffic_volume
0,01,296,1461509905,AL,Alabama,4.937533e+06
1,02,210,318138924,AK,Alaska,1.514947e+06
2,04,526,5600393206,AZ,Arizona,1.064714e+07
3,05,115,278436058,AR,Arkansas,2.421183e+06
4,06,570,6670057776,CA,California,1.170186e+07
5,08,212,964145461,CO,Colorado,4.547856e+06
6,09,75,500556105,CT,Connecticut,6.674081e+06
7,10,168,534104309,DE,Delaware,3.179192e+06
8,11,4,59205437,DC,District of Columbia,1.480136e+07
9,12,545,4412694931,FL,Florida,8.096688e+06


In [99]:
fig = go.Figure()

fig.add_trace(go.Choropleth(
    locations=merge_station_state.fips_state_abb,
    z=merge_station_state["annual_traffic_volume"],
    locationmode="USA-states",
    colorscale="Reds",
    autocolorscale=False,
    colorbar_title="Billions",
    text=merge_station_state.fips_state_full,
    marker_line_color="white",
))

fig.update_layout(
    title="2015 US Traffic Volume by State",
    geo = dict(
        scope="usa", # Limit map scope to USA
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor="rgb(255, 255, 255)"),
    )

In [97]:
merge_station_state.sort_values("annual_traffic_volume", ascending=False).head()

,fips_state_code,num_unique_stations,annual_traffic_volume,fips_state_abb,fips_state_full,norm_annual_traffic_volume
46,51,961,9056241744,VA,Virginia,9.423769e+06
4,06,570,6670057776,CA,California,1.170186e+07
2,04,526,5600393206,AZ,Arizona,1.064714e+07
9,12,545,4412694931,FL,Florida,8.096688e+06
10,13,446,3094751035,GA,Georgia,6.938904e+06


In [98]:
merge_station_state.sort_values("norm_annual_traffic_volume", ascending=False).head()

,fips_state_code,num_unique_stations,annual_traffic_volume,fips_state_abb,fips_state_full,norm_annual_traffic_volume
8,11,4,59205437,DC,District of Columbia,1.480136e+07
39,44,184,2441569023,RI,Rhode Island,1.326940e+07
4,06,570,6670057776,CA,California,1.170186e+07
40,45,131,1511178572,SC,South Carolina,1.153571e+07
2,04,526,5600393206,AZ,Arizona,1.064714e+07


In [101]:
fig = go.Figure()

fig.add_trace(go.Choropleth(
    locations=merge_station_state.fips_state_abb,
    z=merge_station_state["norm_annual_traffic_volume"],
    locationmode="USA-states",
    colorscale="Reds",
    autocolorscale=False,
    colorbar_title="Billions",
    text=merge_station_state.fips_state_full,
    marker_line_color="white",
))

fig.update_layout(
    title="2015 US Traffic Volume by State",
    geo = dict(
        scope="usa", # Limit map scope to USA
        projection=go.layout.geo.Projection(type = 'albers usa'),
        showlakes=True, # lakes
        lakecolor="rgb(255, 255, 255)"),
    )